In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
    """
)

class Classification(BaseModel):
    sentiment: str = Field(...,
        description="The sentiment of the text",
        enum=["happy","neutral","sad"]
    )
    aggressiveness: int = Field(...,
        description="How aggressive the text is on a scale from 1 to 10",
        enum=[1,2,3,4,5]
    )
    language: str = Field(...,
        description="The language the text is written in", 
        enum=["spanish","english","french","german","italian"]
    )

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

In [6]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos"
prompt = tagging_prompt.invoke({"input":inp})
response = llm.invoke(prompt)

print(response)

sentiment='happy' aggressiveness=1 language='spanish'


In [7]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='sad', aggressiveness=5, language='spanish')

In [8]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='english')